# *FEniCS tutorial:* Diffusion of a Gaussian hill in two dimensions

In this demo, we solve the two-dimensional diffusion equation with Dirichlet
bounadary conditions $u_D$ and source term $f = 0$. The initial condition,
$u_0$, we define as a Gaussian hill.

$$
\begin{align}
  u'   &= \nabla^2 u + f \quad\text{in the unit square} \\
  u    &= u_D  \hphantom{u+f}\quad\text{on the boundary} \\
  u    &= u_0  \hphantom{u+f}\quad\;\text{at $t = 0$} \\
\end{align}
$$
with
$$
\begin{align}
  u_D  &= 0 \\
  u_0  &= \exp\bigl(-a(x^2 + y^2)\bigr) \\
  f    &= 0
\end{align}
$$

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time as tm

# import os
# import sys
# import re
# from IPython.display import Image
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

In [ ]:
from fenics import *

In [ ]:
def minmax(it):
    min = max = None
    for val in it:
        if min is None or val < min:
            min = val
        if max is None or val > max:
            max = val
    return min, max

In [ ]:
## set up the problem and define the main simulation function, evolve()

# simulation parameters
# -- time step
T = 0.25          # total simulation time
n_steps = 50      # number of time steps
dt = T / n_steps  # size of time step
nip = 5           # number of intervals between plots
# -- mesh density
nx = ny = 30      # mesh density along axes

# create mesh and define function space
mesh = RectangleMesh(Point(-2, -2), Point(2, 2), nx, ny)
V = FunctionSpace(mesh, 'P', 1)

# define boundary condition
def boundary(x, on_boundary):
    return on_boundary

bc = DirichletBC(V, Constant(0), boundary)

# define initial value
u_0 = Expression('exp(-a*pow(x[0], 2) - a*pow(x[1], 2))',
                 degree=2, a=10)
u_n = interpolate(u_0, V)

u_min = 0.
u_max = 1.

# define variational problem
u = TrialFunction(V)
v = TestFunction(V)

f = Constant(0)
F = u*v*dx + dt*dot(grad(u), grad(v))*dx - (u_n + dt*f)*v*dx
a, L = lhs(F), rhs(F)

# define time-evolution function
def evolve():

    # report initial state
    yield u_n

    # time-stepping
    u = Function(V)
    t = 0
    for n in range(1, n_steps + 1):

        # update current time
        t += dt

        # compute solution
        solve(a == L, u, bc)

        # report current state
        if n % nip == 0:
            yield u

        # update previous solution
        u_n.assign(u)

In [ ]:
n_rows = 3
n_cols = 5
fig_wd = 15
# default sizing here yields unit aspect ratio
plt.figure(figsize = (fig_wd, fig_wd * n_rows // n_cols))

idx = 0
for u in evolve():
    idx += 1
    plt.subplot(n_rows, n_cols, idx)
    plot(u, vmin=u_min, vmax=u_max)

In [ ]:
# ranges
xmin, xmax = minmax(mesh.coordinates()[:,0])
ymin, ymax = minmax(mesh.coordinates()[:,1])

# default sizing here yields unit aspect ratio
plt.figure(figsize = (8,5))

tol = 3.e-3  # avoid hitting points outside the domain
xv = np.linspace(xmin * (1 - tol), xmax * (1 - tol), 51)
pts = [(x_, 0) for x_ in xv]

idx = 0
for u in evolve():
    idx += 1
    ux = np.array([u(pt) for pt in pts])
    plt.plot(xv, ux, lw=1)
plt.plot(xv, ux, 'k', lw=1)
plt.show()

In [ ]:
mesh